# Agents that can code

We are now going to start using chatGPT4o because it can code better than chatGPT3.5. But remember that chatGPT4o is more expensive per token than chatGPT3.5 so keep an eye on your usage here: https://platform.openai.com/usage

> **From now on, we will not execute the code in the Jupyter notebook, this is because of two reasons:**
> 1. We are going to ask our agents to write code and execute it for us. Executing code by an agent inside a Jupyter notebook can be tricky and requires additional configuration. It also depends of the OS being used. To avoid dealing with these issues, we will simply move on to executing scripts instead of notebooks
> 2. We are going to start preparing our code to be able to deploy it on the cloud later, so that we can share a link towards our Agents based app. It is not possible to deploy notebooks, so we are going to start using scripts to get ready for that.
> We are still going to use notebooks to describe and explain our code, but do not execute the code here, instead use the script version of this code.

In [ ]:
# We'll always have to start by creating a llm_config object to configure our agents
llm_config = {
    "model": "gpt-4o", 
    "api_key": "sk-proj-GjIUEqAKI2j04dlC18rZT3BlbkFJnLTC5AeosFLwmSAxKzNU"
    }

## Command Line Executor

This time, we are going to need to use a new tool from autogen. This tool will allow us to execute code locally on our machine. There are two version of this tool, one that relies on Docker and that will create a virtualized version of a python machine to execute the code; and another one that executes the code locally directly on your machine as we've been doing for this class. The Docker version is more secure, because the code always gets executed within a virtual machine and if the LLM generates malicious code, you will not be executing it directly on your machine. But in the context of this class, we will be generating simple code that we will review each time so we will rely on the simpler version to setup that executes code locally.

Let's import this class from autogen and let's setup a local executor environment for our agents.
We are going to specify that if the code hangs and nothing happens during 60 seconds we will interrupt it and that we want all the code to execute and provide results in the `coding` folder:

In [ ]:
## making a Command Line Executor
from autogen.code_utils import create_virtual_env
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

venv_dir = ".env_llm"     #virtual environ directory name
                           ##the visual environ that we'll not be using but only the agents will be
venv_context = create_virtual_env(venv_dir)

executor = LocalCommandLineCodeExecutor(
    virtual_env_context=venv_context,
    timeout=200,            ## it is the number of seconds after which we will interrupt our agent because there can be a problem if it is taking too long
    work_dir="coding",         ## name of the new folder that will be created to store the python scripts and other scripts
)
print(
    executor.execute_code_blocks(code_blocks=[CodeBlock(language="python", code="import sys; print(sys.executable)")])
)

## Code executor agent

We are now going to create two agents:
1. A **code_writer_agent**: this agent is our engineer, it will rely on chatGPT to generate python code (other languages are currently not supported by autogen) to accomplish the requested task.
This agent is not a `ConverseableAgent`, it is one of its subcategories, the `AssistantAgent`. The `AssistantAgent` is designed to write code.

In [ ]:
from autogen import AssistantAgent

# Agent that writes code
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

This agent does not execute code (`code_execution_config=False`) and we do not need to specify a system prompt message for this agent because it already has a default one that is designed to suggest python code when it can help. We can see the default system prompt message odf this type of agents:

In [ ]:
code_writer_agent_system_message = code_writer_agent.system_message
print(code_writer_agent_system_message)

2. Our second type of agent is a **code_executor_agent**: this agent does not use an LLM (`llm_config=False`), this agent will simply execute code it was provided, locally in the `coding` folder, and send back the result of that code as a reply.  
This agent needs a `code_execution_config` to be able to execute code, and we will provide it with the one we defined earlier and that was called `executor` (`code_execution_config={"executor": executor}`). We will also give it a default auto reply that it will use if the user does not have any additional details to provide. Basically, each time this agent will execute some code, it will request feedback from the user, you, and if the user does not provide any feedback, it will use the auto reply.

In [ ]:
from autogen import ConversableAgent

# Code executor agent
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

## Coding task

We will now ask our agents to perform a task that requires coding. The task will be provided to the Code Writer agent by the Code Executor agent, the Code Writer agent will then propose a code that should fulfill the task, the Code Executor agent will then execute that code and report back the resutl to the Code Writer. There might be several exchanges between these two agents until they accomplish the task.  
For example, if a code error is encountered, the error will be reported to the Code Writer who'll propose a corrected version of the code.

Let's prepare a simple task to test our code execution scheme. We want our agents to generate a plot that shows the price evolution of two assets, NVDA and BTC for the last 4 years in a way that makes it easy for us to compare them. We will also add the 50 weeks moving average. Let's write it:

In [ ]:
import datetime

today = datetime.datetime.now().date()

message = f"Today is {today}. "\
"Create a plot showing the normalized price of NVDA and BTC-USD for the last 4 years "\
"with their 50 weeks moving average. "\
"Make sure the code is in a markdown code block, save the figure"\
" to a file asset_analysis.png and show it. Provide all the code necessary in a single python bloc. "\
"Re-provide the code block that needs to be executed with each of your messages. "\
"If python packages are necessary to execute the code, provide a markdown "\
"sh block with only the command necessary to install them and no comments."

During the execution, the executor might (and will most likely will especially if it is the first time) encounter some errors. For example, if you did not install the modules required by the code writer, you will get an error. You will most likely have to go install these packages yourself in your virtual environment, as we did when we installed `openai`. The agents might also just install the missing moduels themselves.

For this task, the code writer will most likely require `matplotlib` and maybe (not necessarily) `pandas`. The LLMs will however tell you how to install missing packages as they will be able to interprete the output of the code and tell you how to fix it. 

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message
)

The code will now execute.

Take the time to observe the interaction between the agents. Before executing the code provided by the Code Writer, you will be requested to provide feedback. That is an important safety feature. Always review the code written by an agent before executing it, you never know, there might be some malicious content. You can also provide feedback if you take something is wrong with the code. If not, you can just press Enter to let the Code Executor go ahead and execute the code.